In [1]:
import os
import io
import json
import pandas as pd
from typing_extensions import TypedDict, Annotated
from contextlib import redirect_stdout

# LangChain 관련 라이브러리
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

# SQLite 관련 라이브러리
import sqlite3
from langchain_community.utilities import SQLDatabase
import tempfile

# API 키 설정 (실제 사용시 환경 변수 또는 .env 파일 사용 권장)
os.environ["OPENAI_API_KEY"] = "OPENAI-API-KEY"

####################################
# 1. 도서관 관련 테이블 DDL 정의
####################################

library_schema = """
-- 도서 테이블
CREATE TABLE books (
    book_id VARCHAR(10) PRIMARY KEY,  -- 도서ID (예: B001)
    title VARCHAR(200) NOT NULL,      -- 도서명
    author VARCHAR(100) NOT NULL,     -- 저자
    publisher VARCHAR(100) NOT NULL,  -- 출판사
    publication_year INT NOT NULL,    -- 출판년도
    isbn VARCHAR(20) NOT NULL,        -- ISBN
    category VARCHAR(50) NOT NULL,    -- 분류
    location VARCHAR(20) NOT NULL,    -- 위치(서가)
    status VARCHAR(20) NOT NULL,      -- 상태 (대출가능/대출중/예약중/분실)
    total_copies INT NOT NULL,        -- 총 권수
    available_copies INT NOT NULL     -- 이용 가능 권수
);

-- 회원 테이블
CREATE TABLE members (
    member_id VARCHAR(10) PRIMARY KEY, -- 회원ID (예: M001)
    name VARCHAR(50) NOT NULL,        -- 회원명
    birth_date DATE NOT NULL,         -- 생년월일
    gender CHAR(1) NOT NULL,          -- 성별 (M/F)
    phone VARCHAR(20) NOT NULL,       -- 전화번호
    email VARCHAR(100) NOT NULL,      -- 이메일
    address VARCHAR(200) NOT NULL,    -- 주소
    membership_date DATE NOT NULL,    -- 가입일자
    membership_type VARCHAR(20) NOT NULL, -- 회원유형 (일반/학생/교사/우대)
    membership_status VARCHAR(20) NOT NULL -- 회원상태 (활성/중지/만료)
);

-- 대출 테이블
CREATE TABLE loans (
    loan_id VARCHAR(10) PRIMARY KEY,  -- 대출ID (예: L001)
    book_id VARCHAR(10) NOT NULL,     -- 도서ID
    member_id VARCHAR(10) NOT NULL,   -- 회원ID
    loan_date DATE NOT NULL,          -- 대출일자
    due_date DATE NOT NULL,           -- 반납예정일
    return_date DATE,                 -- 실제반납일
    status VARCHAR(20) NOT NULL,      -- 상태 (대출중/반납완료/연체)
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id)
);

-- 예약 테이블
CREATE TABLE reservations (
    reservation_id VARCHAR(10) PRIMARY KEY, -- 예약ID (예: R001)
    book_id VARCHAR(10) NOT NULL,      -- 도서ID
    member_id VARCHAR(10) NOT NULL,    -- 회원ID
    reservation_date DATE NOT NULL,    -- 예약일자
    expiry_date DATE NOT NULL,         -- 예약만료일
    status VARCHAR(20) NOT NULL,       -- 상태 (대기중/취소됨/완료됨)
    FOREIGN KEY (book_id) REFERENCES books(book_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id)
);

-- 연체 테이블
CREATE TABLE overdue (
    overdue_id VARCHAR(10) PRIMARY KEY, -- 연체ID (예: O001)
    loan_id VARCHAR(10) NOT NULL,      -- 대출ID
    member_id VARCHAR(10) NOT NULL,    -- 회원ID
    book_id VARCHAR(10) NOT NULL,      -- 도서ID
    overdue_date DATE NOT NULL,        -- 연체시작일
    fine_amount DECIMAL(10,2) NOT NULL, -- 연체료
    payment_status VARCHAR(20) NOT NULL, -- 납부상태 (미납/납부완료)
    FOREIGN KEY (loan_id) REFERENCES loans(loan_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);

-- 직원 테이블
CREATE TABLE staff (
    staff_id VARCHAR(10) PRIMARY KEY,  -- 직원ID (예: S001)
    name VARCHAR(50) NOT NULL,         -- 직원명
    position VARCHAR(50) NOT NULL,     -- 직책
    department VARCHAR(50) NOT NULL,   -- 부서
    phone VARCHAR(20) NOT NULL,        -- 전화번호
    email VARCHAR(100) NOT NULL,       -- 이메일
    hire_date DATE NOT NULL            -- 입사일
);

-- 이벤트 테이블
CREATE TABLE events (
    event_id VARCHAR(10) PRIMARY KEY,  -- 이벤트ID (예: E001)
    title VARCHAR(200) NOT NULL,       -- 이벤트명
    description TEXT NOT NULL,         -- 설명
    event_date DATE NOT NULL,          -- 개최일
    start_time TIME NOT NULL,          -- 시작시간
    end_time TIME NOT NULL,            -- 종료시간
    location VARCHAR(100) NOT NULL,    -- 장소
    max_participants INT NOT NULL,     -- 최대참가자수
    current_participants INT NOT NULL, -- 현재참가자수
    event_status VARCHAR(20) NOT NULL  -- 상태 (예정/진행중/완료/취소)
);

-- 이벤트 참가자 테이블
CREATE TABLE event_participants (
    event_id VARCHAR(10) NOT NULL,    -- 이벤트ID
    member_id VARCHAR(10) NOT NULL,   -- 회원ID
    registration_date DATE NOT NULL,  -- 등록일자
    attendance_status VARCHAR(20) NOT NULL, -- 참석상태 (등록/참석/불참)
    PRIMARY KEY (event_id, member_id),
    FOREIGN KEY (event_id) REFERENCES events(event_id),
    FOREIGN KEY (member_id) REFERENCES members(member_id)
);
"""

####################################
# 2. 도서관 샘플 데이터
####################################

# 도서 샘플 데이터
sample_books = """
INSERT INTO books VALUES('B001', '해리 포터와 마법사의 돌', 'J.K. 롤링', '문학수첩', 2000, '9788983920683', '소설/판타지', 'A-1-1', '대출가능', 5, 3);
INSERT INTO books VALUES('B002', '어린 왕자', '생텍쥐페리', '열린책들', 2015, '9788932917245', '소설/고전', 'A-1-2', '대출가능', 3, 2);
INSERT INTO books VALUES('B003', '1984', '조지 오웰', '민음사', 2012, '9788937834684', '소설/고전', 'A-2-1', '대출중', 2, 0);
INSERT INTO books VALUES('B004', '데미안', '헤르만 헤세', '민음사', 2009, '9788937460449', '소설/고전', 'A-2-2', '대출가능', 3, 1);
INSERT INTO books VALUES('B005', '코스모스', '칼 세이건', '사이언스북스', 2006, '9788983711892', '과학', 'B-1-1', '대출가능', 2, 2);
INSERT INTO books VALUES('B006', '사피엔스', '유발 하라리', '김영사', 2015, '9788934972464', '역사/인류학', 'B-1-2', '대출중', 4, 0);
INSERT INTO books VALUES('B007', '정의란 무엇인가', '마이클 샌델', '김영사', 2014, '9788934972815', '철학', 'B-2-1', '대출가능', 3, 1);
INSERT INTO books VALUES('B008', '클린 코드', '로버트 C. 마틴', '인사이트', 2013, '9788966260959', '컴퓨터/IT', 'C-1-1', '대출가능', 2, 1);
INSERT INTO books VALUES('B009', '파이썬 프로그래밍', '마크 루츠', '에이콘', 2016, '9788960771758', '컴퓨터/IT', 'C-1-2', '대출중', 3, 0);
INSERT INTO books VALUES('B010', '이것이 자바다', '신용권', '한빛미디어', 2017, '9788968481475', '컴퓨터/IT', 'C-2-1', '대출가능', 4, 2);
"""

# 회원 샘플 데이터
sample_members = """
INSERT INTO members VALUES('M001', '김철수', '1985-03-15', 'M', '010-1234-5678', 'kim@example.com', '서울시 강남구', '2022-01-10', '일반', '활성');
INSERT INTO members VALUES('M002', '이영희', '1990-07-20', 'F', '010-2345-6789', 'lee@example.com', '서울시 서초구', '2022-02-15', '학생', '활성');
INSERT INTO members VALUES('M003', '박지민', '1982-12-10', 'M', '010-3456-7890', 'park@example.com', '서울시 송파구', '2022-03-20', '교사', '활성');
INSERT INTO members VALUES('M004', '최수진', '1995-05-25', 'F', '010-4567-8901', 'choi@example.com', '서울시 마포구', '2022-04-10', '학생', '활성');
INSERT INTO members VALUES('M005', '정민준', '1975-09-30', 'M', '010-5678-9012', 'jung@example.com', '서울시 영등포구', '2022-05-05', '일반', '중지');
"""

# 대출 샘플 데이터
sample_loans = """
INSERT INTO loans VALUES('L001', 'B003', 'M001', '2024-03-20', '2024-04-03', NULL, '대출중');
INSERT INTO loans VALUES('L002', 'B006', 'M002', '2024-03-25', '2024-04-08', NULL, '대출중');
INSERT INTO loans VALUES('L003', 'B009', 'M003', '2024-03-28', '2024-04-11', NULL, '대출중');
INSERT INTO loans VALUES('L004', 'B001', 'M004', '2024-03-15', '2024-03-29', '2024-03-28', '반납완료');
INSERT INTO loans VALUES('L005', 'B005', 'M001', '2024-03-10', '2024-03-24', '2024-03-23', '반납완료');
INSERT INTO loans VALUES('L006', 'B007', 'M005', '2024-03-05', '2024-03-19', '2024-03-25', '연체');
"""

# 예약 샘플 데이터
sample_reservations = """
INSERT INTO reservations VALUES('R001', 'B006', 'M003', '2024-03-26', '2024-04-09', '대기중');
INSERT INTO reservations VALUES('R002', 'B003', 'M004', '2024-03-27', '2024-04-10', '대기중');
INSERT INTO reservations VALUES('R003', 'B008', 'M002', '2024-03-15', '2024-03-29', '완료됨');
INSERT INTO reservations VALUES('R004', 'B010', 'M001', '2024-03-10', '2024-03-24', '취소됨');
INSERT INTO reservations VALUES('R005', 'B009', 'M005', '2024-03-28', '2024-04-11', '대기중');
"""

# 연체 샘플 데이터
sample_overdue = """
INSERT INTO overdue VALUES('O001', 'L006', 'M005', 'B007', '2024-03-20', 6000.00, '미납');
"""

# 직원 샘플 데이터
sample_staff = """
INSERT INTO staff VALUES('S001', '박관리', '관장', '관리부', '02-123-4567', 'park@library.com', '2015-01-15');
INSERT INTO staff VALUES('S002', '김사서', '사서', '도서부', '02-123-4568', 'kim@library.com', '2018-03-10');
INSERT INTO staff VALUES('S003', '이행정', '행정직', '행정부', '02-123-4569', 'lee@library.com', '2019-05-20');
INSERT INTO staff VALUES('S004', '최교육', '교육사', '교육부', '02-123-4570', 'choi@library.com', '2020-07-15');
INSERT INTO staff VALUES('S005', '정정보', '정보관리사', 'IT부', '02-123-4571', 'jung@library.com', '2021-09-01');
"""

# 이벤트 샘플 데이터
sample_events = """
INSERT INTO events VALUES('E001', '독서 토론회: 사피엔스', '유발 하라리의 사피엔스를 읽고 토론하는 시간', '2024-04-15', '14:00', '16:00', '도서관 세미나실', 20, 12, '예정');
INSERT INTO events VALUES('E002', '어린이 독서 교실', '어린이를 위한 독서 지도', '2024-04-20', '10:00', '12:00', '도서관 아동실', 15, 10, '예정');
INSERT INTO events VALUES('E003', '작가와의 만남', '신간 도서 소개 및 작가와의 대화', '2024-03-25', '15:00', '17:00', '도서관 강당', 50, 45, '완료');
INSERT INTO events VALUES('E004', '책 읽어주는 할머니', '어린이를 위한 스토리텔링', '2024-03-30', '11:00', '12:00', '도서관 아동실', 20, 18, '취소');
INSERT INTO events VALUES('E005', '코딩 워크샵', '청소년을 위한 기초 코딩 교육', '2024-04-25', '13:00', '15:00', '도서관 컴퓨터실', 15, 5, '예정');
"""

# 이벤트 참가자 샘플 데이터
sample_event_participants = """
INSERT INTO event_participants VALUES('E001', 'M001', '2024-03-20', '등록');
INSERT INTO event_participants VALUES('E001', 'M003', '2024-03-21', '등록');
INSERT INTO event_participants VALUES('E002', 'M002', '2024-03-25', '등록');
INSERT INTO event_participants VALUES('E003', 'M004', '2024-03-15', '참석');
INSERT INTO event_participants VALUES('E003', 'M005', '2024-03-16', '불참');
"""

# 전체 DDL과 샘플 데이터 합치기
library_complete_schema = library_schema + sample_books + sample_members + sample_loans + sample_reservations + sample_overdue + sample_staff + sample_events + sample_event_participants

####################################
# 3. SQLite 데이터베이스 설정
####################################

# 임시 SQLite 데이터베이스 생성
def create_library_db():
    db_file = os.path.join(tempfile.gettempdir(), "library_semi.db")
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # 기존 테이블 삭제 (이미 존재하는 경우)
    cursor.executescript("""
    DROP TABLE IF EXISTS events;
    DROP TABLE IF EXISTS book_purchases;
    DROP TABLE IF EXISTS librarians;
    DROP TABLE IF EXISTS overdue_records;
    DROP TABLE IF EXISTS book_status;
    DROP TABLE IF EXISTS reservations;
    DROP TABLE IF EXISTS loans;
    DROP TABLE IF EXISTS members;
    DROP TABLE IF EXISTS books;
    """)

    # DDL 및 샘플 데이터 실행
    cursor.executescript(library_complete_schema)

    conn.commit()
    conn.close()

    # LangChain SQL 데이터베이스 인스턴스 반환
    return SQLDatabase.from_uri(f"sqlite:///{db_file}")

# 데이터베이스 생성 실행
db = create_library_db()
print("데이터베이스가 성공적으로 생성되었습니다.")
print(f"사용 가능한 테이블: {db.get_usable_table_names()}")

# 테이블 내용 확인 예시
books_example = db.run("SELECT * FROM books LIMIT 3;")
print(f"도서 테이블 샘플: {books_example}")

####################################
# 4. Text-to-SQL 시스템 구현
####################################

# LLM 모델 초기화 (API 키 설정 필요)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SQL 쿼리 생성 함수
def generate_sql(question):
    """사용자 질문으로부터 SQL 쿼리 생성"""
    system_prompt = """
    당신은 도서관 데이터베이스 SQL 전문가입니다.
    사용자의 질문을 받아서 해당 질문에 답하기 위한 SQL 쿼리를 생성해야 합니다.

    다음은 데이터베이스 스키마입니다:
    {schema}

    사용 가능한 테이블:
    {table_names}

    주어진 질문에 대한 정확한 SQL 쿼리를 생성하세요.
    SQL 쿼리만 반환하고 다른 설명은 포함하지 마세요.
    쿼리는 SQLite 문법을 사용해야 합니다.

    중요: 사용자 질문에 이전 대화 내용이 포함된 경우, 맥락에 따라 정확한 SQL 쿼리를 생성하세요.
    예를 들어 "그 책의 다른 대출 기록도 보여줘"와 같은 질문이 있으면,
    이전 대화에서 언급된 책을 참조하여 적절한 SQL 쿼리를 생성해야 합니다.

    만약 질문이 데이터베이스 쿼리로 해결할 수 없는 경우(예: 인사말, 일반적인 대화, 데이터베이스와 무관한 질문),
    'SQL_QUERY_NOT_APPLICABLE'이라고 답변하세요.
    """

    human_prompt = "{question}"

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", human_prompt),
    ])

    filled_prompt = prompt.invoke({
        "schema": library_schema,
        "table_names": ", ".join(db.get_usable_table_names()),
        "question": question
    })

    response = llm.invoke(filled_prompt.to_string())
    # SQL 쿼리만 추출 (```sql과 ``` 제거)
    query = response.content
    if "```sql" in query:
        query = query.split("```sql")[1].split("```")[0].strip()
    elif "```" in query:
        query = query.split("```")[1].split("```")[0].strip()

    return query

# SQL 쿼리 실행 함수
def execute_sql(query):
    """SQL 쿼리 실행"""
    try:
        result = db.run(query)
        return result
    except Exception as e:
        return f"에러 발생: {str(e)}"

# 자연어 답변 생성 함수
def generate_answer(question, query, result):
    """쿼리 결과를 바탕으로 사용자 질문에 자연어로 답변"""
    prompt = f"""
    다음은 사용자의 질문, SQL 쿼리, 그리고 그 결과입니다:

    질문: {question}
    SQL 쿼리: {query}
    결과: {result}

    이 정보를 바탕으로 사용자 질문에 답변해주세요. SQL 쿼리나 기술적인 세부사항은 언급하지 말고,
    일반인이 이해할 수 있는 자연어로 친절하게 답변해주세요.

    답변은 대화 맥락을 고려해야 합니다. 사용자가 이전 질문의 결과를 참조하거나 추가 정보를 요청하는 경우,
    일관된 대화 흐름을 유지하며 답변해주세요.
    """
    response = llm.invoke(prompt)
    return response.content

# 대화 컨텍스트 관리 클래스
class ConversationContext:
    def __init__(self):
        self.history = []  # 대화 이력 저장 [(질문, 쿼리, 결과, 답변), ...]

    def add_interaction(self, question, query, result, answer):
        self.history.append((question, query, result, answer))

    def get_context_str(self):
        if not self.history:
            return ""

        context = "이전 대화 내용:\n"
        for i, (q, _, _, a) in enumerate(self.history[-3:]):  # 최근 3개 대화만 컨텍스트로 사용
            context += f"질문 {i+1}: {q}\n"
            context += f"답변 {i+1}: {a}\n"
        return context

# 대화 컨텍스트 인스턴스 생성
conversation_context = ConversationContext()

# 전체 처리 과정 함수 (컨텍스트 포함)
def process_question(question, use_context=True):
    """질문 처리 전체 프로세스 (대화 컨텍스트 포함)"""
    # 이전 대화 컨텍스트 가져오기
    context_str = conversation_context.get_context_str() if use_context else ""

    # 컨텍스트가 있는 경우 질문에 추가
    if context_str:
        enhanced_question = f"{context_str}\n새로운 질문: {question}"
    else:
        enhanced_question = question

    query = generate_sql(enhanced_question)

    # SQL 쿼리로 해결할 수 없는 질문 처리
    if query.strip() == "SQL_QUERY_NOT_APPLICABLE":
        # 일반적인 답변 생성
        prompt = f"""
        다음은 사용자의 질문입니다:

        질문: {question}

        이 질문은 도서관 데이터베이스 쿼리로 직접 해결할 수 없는 것으로 판단됩니다.
        사용자에게 친절하게 답변해주세요. 만약 도서관 관련 일반 정보를 요청한 것이라면,
        일반적인 정보를 제공하고, 데이터베이스를 통해 더 구체적인 정보를 얻으려면 어떻게 질문해야 할지 안내해주세요.
        """

        response = llm.invoke(prompt)
        answer = response.content

        # 대화 내용 저장
        conversation_context.add_interaction(question, "적용 불가", "쿼리 없음", answer)

        return "적용 불가", "SQL 쿼리로 해결할 수 없는 질문입니다.", answer

    # 일반적인 SQL 쿼리 실행
    result = execute_sql(query)
    answer = generate_answer(question, query, result)

    # 대화 내용 저장
    conversation_context.add_interaction(question, query, result, answer)

    return query, result, answer

####################################
# 5. Gradio UI 구현 및 실행
####################################

# Gradio UI 함수
def ui_process_question(question, history):
    query, result, answer = process_question(question, use_context=True)

    # UI에 표시할 내용 처리
    if query == "적용 불가":
        display_query = "SQL 쿼리 해당 없음"
        display_result = "데이터베이스 조회 없음"
    else:
        display_query = query
        display_result = result

    return "", history + [(question, answer)], display_query, display_result

# 대화 초기화 함수
def clear_conversation():
    # 대화 이력 초기화
    conversation_context.history = []
    return [], "", ""

# Gradio 인터페이스 구성 및 실행
def launch_gradio_interface():
    # UI 시작 전에 대화 컨텍스트 초기화
    conversation_context.history = []
    print("대화 컨텍스트가 초기화되었습니다. 깨끗한 상태에서 시작합니다.")

    with gr.Blocks(css="footer {visibility: hidden}") as demo:
        gr.Markdown("# 도서관 데이터베이스 질의 시스템")
        gr.Markdown("도서관 데이터에 관한 도서, 회원, 대출, 예약, 이벤트 등에 대한 질문을 해보세요.")

        with gr.Row():
            with gr.Column(scale=2):
                chatbot = gr.Chatbot(height=400)
                msg = gr.Textbox(label="질문을 입력하세요", placeholder="예: 현재 대출 가능한 책은 몇 권인가요?")
                clear = gr.Button("대화 초기화")
            with gr.Column(scale=1):
                sql_output = gr.Textbox(label="생성된 SQL 쿼리", lines=4)
                result_output = gr.Textbox(label="쿼리 결과", lines=8)

        msg.submit(ui_process_question, inputs=[msg, chatbot], outputs=[msg, chatbot, sql_output, result_output])
        clear.click(clear_conversation, outputs=[chatbot, sql_output, result_output])

        gr.Markdown("""
        ### 예시 질문:
        - 현재 대출 가능한 도서는 몇 권인가요?
        - 가장 인기있는 도서 카테고리는 무엇인가요?
        - 김철수 회원이 현재 대출중인 도서 목록을 보여주세요.
        - 연체된 도서는 몇 권인가요?
        - 다음 달에 예정된 이벤트는 무엇인가요?
        - 가장 많은 책을 대출한 회원은 누구인가요?
        - 컴퓨터/IT 카테고리의 도서 목록을 보여주세요.
        - 현재 가장 많이 예약된 도서는 무엇인가요?
        """)

    # Gradio 인터페이스 실행
    demo.launch(share=True)

# 예시 질문 테스트
test_question = "현재 대출 가능한 도서는 몇 권인가요?"
query, result, answer = process_question(test_question)

print("\n--- 테스트 결과 ---")
print(f"질문: {test_question}")
print(f"생성된 SQL 쿼리: {query}")
print(f"쿼리 결과: {result}")
print(f"생성된 답변: {answer}")

# Gradio 인터페이스 실행
launch_gradio_interface()

ModuleNotFoundError: No module named 'langchain'

In [3]:
import os
import tempfile
import sqlite3

# 데이터베이스 파일 경로 확인
db_file = os.path.join(tempfile.gettempdir(), "library_semi.db")

# 데이터베이스 연결 및 테이블 목록 확인
if os.path.exists(db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("삭제 전 데이터베이스의 테이블 목록:", tables)
    conn.close()
    
    # 파일 삭제
    os.remove(db_file)
    print(f"{db_file} 파일이 삭제되었습니다.")
else:
    print(f"{db_file} 파일이 존재하지 않습니다.")

삭제 전 데이터베이스의 테이블 목록: [('staff',), ('event_participants',), ('books',), ('members',), ('loans',), ('reservations',), ('overdue',)]
/tmp/library_semi.db 파일이 삭제되었습니다.
